## 将实例化的逻辑表达式转化为自然语言

In [12]:
from typing import List
from typing import Optional
from typing import Dict

import os
import json
from pathlib import Path

import httpx
from openai import OpenAI

import requests

import time
import re
from tqdm import tqdm

## 逻辑表达式转化为自然语言

In [2]:
# 文件路径
file_path = "prompts/DataFountain/nl.txt"

# 请求的 URL
url = "https://api.moonshot.cn/v1/files"

headers = {
    'Authorization': 'sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw'
}

# 准备文件数据
files = {'file': open(file_path, 'rb')}

# 发起 POST 请求
response = requests.post(url, headers=headers, files=files)

# 打印响应内容
print(response.status_code)
print(response.text)

200
{"id":"ct7t5pb5cfusk9k5e3og","object":"file","bytes":6097,"created_at":1733284581,"filename":"nl.txt","purpose":"file-extract","status":"ok","status_details":""}


In [3]:
client = OpenAI(
    api_key="sk-gNToXJPO4Ih8NDZnDloSXR780dQDEZ2Epu49wcVA1sWb4Ecw",
    base_url="https://api.moonshot.cn/v1",
)

file_object = client.files.create(file=Path("prompts/DataFountain/nl.txt"), purpose="file-extract")
file_content = client.files.content(file_id=file_object.id).text

In [4]:
with open('data/DF/instance_predicates_v3.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [11]:
print(len(data))

751


In [25]:
def extract_json(text):
    # 使用正则表达式匹配并提取 JSON 数据（假设 JSON 数据用 `{` 开头和 `}` 结尾）
    json_data = re.search(r'\[.*\]', text, re.DOTALL)

    # 检查是否找到匹配的 JSON 数据
    if json_data:
        json_string = json_data.group(0)

        # 将 JSON 数据解析为字典
        data = json.loads(json_string)
        return data
    else:
        print("未找到 JSON 数据。")

In [11]:
print(extract_json(data[0]))

[
    {
        "instance_predicates": [
            "c=“甲苯泄漏” — 表示危险化学品事故为甲苯泄漏",
            "i=“工人受伤” — 表示人员伤害为工人受伤",
            "p=“设备损坏” — 表示财产损失为设备损坏",
            "e=“河流污染” — 表示环境污染为河流污染",
            "Cause=“甲苯泄漏导致” — 表示事故原因为甲苯泄漏导致"
        ],
        "instance_logicexpression": [
            "∀c (ChemicalAccident(“甲苯泄漏”, “工人受伤”, “设备损坏”, “河流污染”) ↔ (Produce(“甲苯泄漏”) ∨ Operate(“甲苯泄漏”) ∨ Store(“甲苯泄漏”) ∨ Transport(“甲苯泄漏”) ∨ Use(“甲苯泄漏”) ∨ Dispose(“甲苯泄漏”)) ∧ (Cause(“甲苯泄漏”, “工人受伤”) ∧ Cause(“甲苯泄漏”, “设备损坏”) ∧ Cause(“甲苯泄漏”, “河流污染”)) ∧ ¬(MiningAccident(“甲苯泄漏”, GasPoisoning) ∨ MiningAccident(“甲苯泄漏”, Explosion) ∨ MiningAccident(“甲苯泄漏”, Blasting))))"
        ]
    },
    {
        "instance_predicates": [
            "c=“硫酸泄漏” — 表示危险化学品事故为硫酸泄漏",
            "i=“居民中毒” — 表示人员伤害为居民中毒",
            "p=“仓库损毁” — 表示财产损失为仓库损毁",
            "e=“土壤污染” — 表示环境污染为土壤污染",
            "Cause=“硫酸泄漏导致” — 表示事故原因为硫酸泄漏导致"
        ],
        "instance_logicexpression": [
            "∀c (ChemicalAccident(“硫酸泄漏”, “

## 使用api转化自然语言

In [6]:
def call_api_with_rate_limit_handling(client, messages, model="moonshot-v1-32k", max_retries=5):
    retries = 0
    while retries < max_retries:
        try:
            completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0,
            )
            return completion
        except Exception as e:
            if '429' in str(e):  # 检查异常消息中是否包含429
                print("Rate limit reached. Retrying in 1 second...")
                time.sleep(1)  # 等待1秒钟
                retries += 1
            else:
                raise  # Re-raise the exception if it's not a 429 error
    raise Exception("Max retries reached. API call failed.")

In [15]:
nl = []

In [16]:
for dicts in tqdm(data, desc='rule转化逻辑表达式进度'):
    # 替换内容
    # 使用正则表达式替换 [[RULES]]
    # content = re.sub(r'\[\[RULES\]\]', instance, file_content)
    # print(content)

    # 使用replace
    content = file_content.replace("[[RULES]]", dicts)

    messages = [
        {
            "role": "system",
            "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。请将内容输出为可解析的json格式。"
        },
        {
            "role": "system",
            "content": content,
        },
        {
            "role": "user",
            "content": "你好，请根据txt内容生成回答。"
        }
    ]

    completion = call_api_with_rate_limit_handling(client, messages, "moonshot-v1-32k", 5)

    nl.append(completion.choices[0].message.content)

rule转化逻辑表达式进度: 100%|██████████| 751/751 [2:47:41<00:00, 13.40s/it]  


In [18]:
print(nl[0])

你好，以下是根据提供的逻辑表达式和谓词描述转换成自然语言的输出：

```json
[
    {
        "natural_language": "如果工厂发生了爆炸事件，并且涉及的是易燃化学品，那么这个事件会导致人员伤害、财产损失和环境污染。同时，这个事件与生产、操作、储存、运输、使用或处置易燃化学品有关，并且不是矿井事故、有毒气体中毒、爆炸事故或爆破事故。"
    },
    {
        "natural_language": "如果化工厂发生了化学品泄漏事件，并且涉及的是有毒化学品，那么这个事件会导致人员伤害、财产损失和环境污染。同时，这个事件与生产、操作、储存、运输、使用或处置有毒化学品有关，并且不是矿井事故、有毒气体中毒、爆炸事故或爆破事故。"
    },
    {
        "natural_language": "如果在运输途中发生了化学品泄漏事件，并且涉及的是腐蚀性化学品，那么这个事件会导致人员伤害、财产损失和环境污染。同时，这个事件与生产、操作、储存、运输、使用或处置腐蚀性化学品有关，并且不是矿井事故、有毒气体中毒、爆炸事故或爆破事故。"
    },
    {
        "natural_language": "如果实验室发生了化学品爆炸事件，并且涉及的是实验用化学品，那么这个事件会导致人员伤害、财产损失和环境污染。同时，这个事件与生产、操作、储存、运输、使用或处置实验用化学品有关，并且不是矿井事故、有毒气体中毒、爆炸事故或爆破事故。"
    }
]
```


In [17]:
with open('data/DF/instance_nl_v1.json', 'w', encoding='utf-8') as f:
    json.dump(nl, f, ensure_ascii=False, indent=4)